In [1]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn import ItemKNN
import pandas as pd

In [2]:
dataset = Dataset()

In [3]:
split_dict = dataset.get_split()

In [4]:
train, train_label = split_dict[TRAIN]

In [5]:
val, val_label = split_dict[VAL]

In [6]:
lead_data = dataset.get_test_leaderboard_sessions()

In [7]:
model = ItemKNN(dataset, topk=1000)

In [8]:
model.compute_similarity_matrix(train)

Done: 100%|██████████| 23691/23691 [00:00<00:00, 31063.35it/s]


In [37]:
recs = model.recommend(
    interactions=val,
    remove_seen=True,
    cutoff=100,
    leaderboard=False
)

In [21]:
recs

,session_id,item_id,score,rank
0,107,6258,0.026263,1
1,107,17928,0.021998,2
2,107,17787,0.021563,3
3,107,12940,0.019449,4
4,107,14480,0.019189,5
...,...,...,...,...
9999995,4440001,15338,0.030493,96
9999996,4440001,7259,0.030382,97
9999997,4440001,9238,0.030342,98
9999998,4440001,2818,0.030196,99


In [22]:
val

,session_id,item_id,date
0,107,16083,2020-01-15 06:05:10.332
1,107,17011,2020-01-15 06:05:18.497
2,107,23135,2020-01-15 06:04:58.297
3,108,4559,2020-06-06 17:13:32.996
4,108,22024,2020-06-06 17:14:10.825
...,...,...,...
477753,4440001,17238,2020-10-30 23:37:20.658
477754,4440001,11916,2020-10-30 23:31:56.607
477755,4440001,12039,2020-10-30 23:36:17.934
477756,4440001,23434,2020-10-30 23:39:55.186


In [28]:
recs

,session_id,item_id,score,rank
0,107,6258,0.026263,1
1,107,17928,0.021998,2
2,107,17787,0.021563,3
3,107,12940,0.019449,4
4,107,14480,0.019189,5
...,...,...,...,...
9999995,4440001,15338,0.030493,96
9999996,4440001,7259,0.030382,97
9999997,4440001,9238,0.030342,98
9999998,4440001,2818,0.030196,99


In [30]:
val[SESS_ID].unique()

array([    107,     108,     124, ..., 4439964, 4439986, 4440001])

In [38]:

merged=pd.merge(recs, val_label, on=[SESS_ID, ITEM_ID], how="right")

In [34]:
merged

,session_id,item_id,score,rank,date
0,107,3370,NaN,NaN,2020-01-15 06:07:23.177
1,108,15639,NaN,NaN,2020-06-06 17:25:15.508
2,124,3126,NaN,NaN,2020-05-18 11:04:15.420
3,171,11806,NaN,NaN,2020-12-18 01:12:30.945
4,324,22049,0.095987,1.0,2020-05-25 15:40:17.987
...,...,...,...,...,...
99995,4439892,7623,0.038059,1.0,2020-03-01 13:52:26.972
99996,4439953,2179,NaN,NaN,2021-04-02 11:31:29.357
99997,4439964,17216,NaN,NaN,2020-03-24 08:18:38.953
99998,4439986,2445,0.037547,20.0,2021-05-13 11:56:37.464


In [39]:
merged["mrr"] = 1/merged["rank"]
merged["mrr"] = merged["mrr"].fillna(0)

In [26]:
len(merged[SESS_ID].unique())

100000

In [27]:
merged["mrr"].sum()/merged[SESS_ID].nunique()

5.432176392206084

In [40]:
merged["mrr"].mean()

0.1525491659209942

In [ ]:
val

In [ ]:
dataset.get_candidate_items().values.squeeze()